In [1]:
import utils

import os, shutil
from datetime import datetime

import sqlalchemy
from sqlalchemy import create_engine, types
from sqlalchemy.dialects.mysql import LONGTEXT

import pandas as pd
from tqdm import tqdm

### Set Variables
---

In [2]:
# Ideally this has to be read in from a configuration file. 
# However, reading from a variable for now.

root_directory = '/home/sree/code/apptio-data/clickstream_data_2016'
log_directory_root = "log"
start_time = str(datetime.now())[:16]

schema_file_name = "schema.txt"
failed_file_name = "failed.txt"
success_file_name = "processed.txt"


hostname="localhost"
dbname="apptio"
uname="sreejith"
pwd="password"
table = "clickthrough_2016_09"


engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=uname, pw=pwd))

In [3]:
# Create the log folder the current run
log_folder = os.path.join(log_directory_root, start_time)
if not os.path.exists(log_folder):
    os.makedirs(log_folder)

In [4]:
gzip_paths = utils.get_gzip_file_paths(root_directory)
schema, successful_paths, failed_paths = utils.find_full_schema(gzip_paths)

205it [00:00, 4797.88it/s]
100%|██████████| 87/87 [04:56<00:00,  3.40s/it]


Note: Every zip file has 36 columns or less. Write the schema to the log directory.

In [5]:
with open(os.path.join(log_folder, schema_file_name), "w") as f:
    f.writelines(["\n" + x for x in schema])
with open(os.path.join(log_folder, success_file_name), "w") as f:
    f.writelines(["\n" + x for x in successful_paths])

if len(failed_paths) > 0:
    with open(os.path.join(log_folder, failed_file_name), "w") as f:
        f.writelines(["\n" + x for x in failed_paths]) 

#### Create a database to store the data from the clickthrough table
---

In [6]:
connection = engine.raw_connection()
cursor = connection.cursor()
table = "events"
command = "DROP TABLE IF EXISTS {};".format(table)
cursor.execute(command)

table = "status"
command = "DROP TABLE IF EXISTS {};".format(table)
cursor.execute(command)

table = "users"
command = "DROP TABLE IF EXISTS {};".format(table)
cursor.execute(command)

connection.commit()

In [7]:
utils.standardize_and_store(successful_paths, schema, table, engine)

100%|██████████| 87/87 [30:25<00:00, 20.98s/it]


### Note: 
IP address and request_datetime can be used as the key to connect users to an event
GUID is not a reliable key since the same GUID is being assigned to different people (with different id and different request_datetime)